In [1]:
import numpy as np

import matplotlib.pyplot as plt
import os
# import glob
import pickle
import tqdm
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib as mlp
import matplotlib.pyplot as plt
mlp.rc("xtick",labelsize=12)
mlp.rc("ytick",labelsize=12)
mlp.rc("axes",labelsize=14)

plt.rcParams['figure.figsize'] = [10,5]
plt.rcParams['figure.dpi'] = 200

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.decomposition import FastICA


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import keras

CURR_DIR = os.getcwd()

In [2]:

# MAIN_DIR = os.path.join(CURR_DIR,"..","..")
MAIN_DIR = CURR_DIR
DATA_DIR = os.path.join(MAIN_DIR,"dataset")
FILE_DIR = os.path.join(MAIN_DIR,"dataset","pickle")
FIG_DIR = os.path.join(MAIN_DIR,"figures")
os.makedirs(FIG_DIR,exist_ok=True)

SPEAKER = ["RL","RN","SR","US"]
MODE = ["mentally","mouthed"]
WORDS = LABELS = ["add","call","go","later","left","reply","right","stop","subtract","you"]

SAMPLING_FREQ = 250 
NUM_CHANNELS = 8


# a function to save plotted figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(FIG_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


###  LOAD DATA

In [3]:
DATASET = os.listdir(FILE_DIR)
DATASET.sort()
print(DATASET)
print("***********************")
print(DATASET[-2])

['ALL_feature_spectral_me.pickle', 'ALL_feature_spectral_mo.pickle', 'ALL_feature_time_me.pickle', 'ALL_feature_time_mo.pickle', 'ALL_feature_time_spectral_me.pickle', 'ALL_feature_time_spectral_mo.pickle', 'RL_feature_spectral_me.pickle', 'RL_feature_spectral_mo.pickle', 'RL_feature_time_me.pickle', 'RL_feature_time_mo.pickle', 'RL_feature_time_spectral_me.pickle', 'RL_feature_time_spectral_mo.pickle', 'RN_feature_spectral_me.pickle', 'RN_feature_spectral_mo.pickle', 'RN_feature_time_me.pickle', 'RN_feature_time_mo.pickle', 'RN_feature_time_spectral_me.pickle', 'RN_feature_time_spectral_mo.pickle', 'SR_feature_spectral_me.pickle', 'SR_feature_spectral_mo.pickle', 'SR_feature_time_me.pickle', 'SR_feature_time_mo.pickle', 'SR_feature_time_spectral_me.pickle', 'SR_feature_time_spectral_mo.pickle', 'US_feature_spectral_me.pickle', 'US_feature_spectral_mo.pickle', 'US_feature_time_me.pickle', 'US_feature_time_mo.pickle', 'US_feature_time_spectral_me.pickle', 'US_feature_time_spectral_mo.pi

In [4]:

# feature_spectral_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[0]),"rb"))
# feature_spectral_mouthed = pickle.load(open(os.path.join(FILE_DIR,DATASET[1]),"rb"))
# feature_time_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[2]),"rb"))
# feature_time_mouthed = pickle.load(open(os.path.join(FILE_DIR,DATASET[3]),"rb"))
feature_time_spectral_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[-2]),"rb"))
# feature_time_spectral_mouthed = pickle.load(open(os.path.join(FILE_DIR,DATASET[-1]),"rb"))

In [6]:
# X = feature_time_spectral_mouthed["data"]
# Y = feature_time_spectral_mouthed["label"]
X = feature_time_spectral_mental["data"]
Y = feature_time_spectral_mental["label"]

print(X.shape)
print(Y.shape)


(1783, 1262, 112)
(1783,)


### LOAD MODEL

In [7]:
# from keras.models import load_model
# model = tf.keras.models.load_model("all_features_all_speakers_mouthed_model_cnn_e15.h5")

model = tf.keras.models.load_model("all_features_all_speakers_mental_model_cnn_e15.h5")

In [8]:
model.input

<tf.Tensor 'conv1d_input:0' shape=(None, 885, 112) dtype=float32>

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 874, 100)          134500    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 437, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 432, 100)          60100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 216, 100)          0         
_________________________________________________________________
flatten (Flatten)            (None, 21600)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               2160100   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

### PREPARE DATA

In [10]:
scalar = StandardScaler()
X = (scalar.fit_transform(X.reshape(X.shape[0],-1),Y)).reshape(X.shape[0],X.shape[1],-1)


In [11]:
encoder = LabelEncoder()

Y = encoder.fit_transform(Y)
print(encoder.classes_)
print("Encoded labels (normal): \n",Y[0:5])

['add' 'call' 'go' 'later' 'left' 'reply' 'right' 'stop' 'subtract' 'you']
Encoded labels (normal): 
 [1 1 1 1 1]


In [12]:
print(X.shape)
print(Y.shape)

(1783, 1262, 112)
(1783,)


### PADDING AND TRIMMING DATA

In [ ]:
# a = np.floor(int((int(X.shape[1])-885)/2))
# b = np.ceil(int((int(X.shape[1])-885)/2))

In [ ]:
# print(a)
# print(b)
len(X[1])

In [13]:
# a =188 #17
# b = 189 #18
a =188
b = 189
temp = []
for i in range(len(X)):
    temp.append(X[i][a:-b,:])
X = np.array(temp)
print(X.shape)

(1783, 885, 112)


In [14]:
model.evaluate(X,Y)

56/56 [==============================] - 5s 81ms/step - loss: 7.0349 - accuracy: 0.0426


[7.034923553466797, 0.04262479022145271]

In [ ]:
model.evaluate(X,Y)

In [ ]:
predictions = np.argmax(model.predict(X))

In [ ]:
print(predictions[0])